# Lab | APIs

In order to use the `Spotify` API (`SpotiPy`), create an account in `Spotify` and follow [these](https://developer.spotify.com/documentation/general/guides/app-settings/) steps. 

## Authentication and initializing the API

Save your client ID and your client secret in your preferred way, and read it or load it into the following variables:

Once you have done it, we will start initializing the API.

In [48]:
import pandas as pd
import json
import spotipy
import functools
from spotipy.oauth2 import SpotifyClientCredentials
import os
from dotenv import load_dotenv

In [49]:
#load env
load_dotenv()
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [50]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET))

## Using the search method

Now, let's use the search method by introducing a "query". For example, let's try searching for "Lady Gaga":

In [51]:
results = sp.search(q='Lady Gaga', limit=50)

In [ ]:
results.keys() # We can see that we only have tracks

In [ ]:
results["tracks"].keys() # Let's check the values

In [ ]:
results["tracks"]["href"] # Query we have searched 

In [ ]:
results["tracks"]["items"] #items (actual tracks)

In [ ]:
results["tracks"]["limit"]#Limit we have chosen

In [ ]:
results["tracks"]["next"] #link to the next page (next 50 tracks)

In [ ]:
results["tracks"]["offset"] # Actual offset (starting point)

In [59]:
results["tracks"]["previous"] #Previous search

In [ ]:
results["tracks"]["total"] # Number of matches

## Exploring the tracks

In [ ]:
results["tracks"]["items"][0] # Explore the first song

In [ ]:
results["tracks"]["items"][0].keys() # We will focus on album, artists, id, name, popularity, type and uri

In [ ]:
# Track artists
results["tracks"]["items"][0]["artists"] 

In [ ]:
# Track artists names
for artist in results["tracks"]["items"][0]["artists"]:
    print(artist["name"])

In [ ]:
# Track ID
results["tracks"]["items"][0]["id"] 

In [ ]:
# Track name
results["tracks"]["items"][0]["name"] 

In [ ]:
# Popularity index
results["tracks"]["items"][0]["popularity"] 

Spotify songs are identified by either a "url", a "uri", or an "id". 

- The `id` is an alphanumeric code, and it's the nuclear part of the identifier.

- The `uri` contains "spotify:track" before the id. An uri is useful because it can be searched manually in the Spotify app.

- The `url` is a link to the song on the Spotify web player.


In [ ]:
results["tracks"]["items"][0]["uri"]

## Exercise 1: Discovering New Music through Your Favorite Artists

**Objective:** 
Uncover new music by exploring the top tracks of your favorite artists and their related artists.

**Instructions:**

1. **List Your Favorite Artists**:
    - Make a list of your three favorite artists and store it in a variable named `artists`.
    - Example: `artists = ["Los Fabulosos Cadillacs", "Manu Chao", "Muchachito Bombo Infierno"]`.

2. **Fetch Top Tracks**:
    - Write a function named `get_top_tracks`.
    - This function should accept an artist's name and return the name of the first 5 top tracks by that artist.
    - Use the function `get_top_tracks` to get the first 5 top tracks for each artist in your `artists` list and store the results in a new list named `top_tracks_list`.

3. **Discover Related Artists**:
    - Write a function named `find_related_artists`.
    - This function should accept an artist's name and return the names of the first 5 artists related to the provided artist.
    - Store the results in a list named `related_artists_list`.

**Challenge:** 
Combine the above steps to create a playlist that includes the top tracks of your favorite artists and the top tracks of the artists related to them.

**Hint Section for 3. **Discover Related Artists**:**

1. **Getting Artist ID**:
    - Remember that every artist on Spotify has a unique identifier: their `id`. To get the related artists, you first need to fetch the ID of the given artist.
    - Consider using the `sp.search` method to query the artist's name. The method requires a `q` parameter, which is your query (in this case, the artist's name). It also has a `limit` parameter, which specifies the number of tracks it returns. In this case, 1 track is enough, since we just want the artist ID. 
    - Each track in the results has an associated 'artists' field. This field is a list containing details about all artists involved in that track.
   - For most tracks, especially those by a single artist, this list will contain one artist. From this artist's details, you can extract the 'id' field, which is the unique identifier for that artist on Spotify.


3. **Fetching Related Artists**:
    - Once you have the artist's ID, you can use another SpotiPy method to fetch related artists. Think about which SpotiPy method allows you to get related artists using an artist's ID. Here is the documentation link: https://spotipy.readthedocs.io/en/2.22.1/. 
    - This method will return a list of related artists. You'll need to extract the relevant details (artist names) from this list.

4. **Iterating for Multiple Artists**:
    - Once you have a function that returns related artists names for one artist, you can use a list comprehension to apply this function to a list of artist names.

5. **Testing**:
    - Always test your function with one artist name first. Once you're confident it works, then apply it to the entire list.

Remember, the key is to break the problem down into manageable steps. Use the SpotiPy documentation as a resource to understand available methods and their return structures.

In [69]:
artists = ["Lhasa De Sala", "Nouvelle Vague", "Herbie Hancock"]

In [ ]:
def get_top_tracks(name, max_songs=5):
    data = sp.search(q=name)
    id = data["tracks"]["items"][0]["artists"][0]["id"]
    tracks = sp.artist_top_tracks(id)
    tracks_names = [track["name"] for track in tracks["tracks"]]
    print(tracks_names)
    return tracks_names[:max_songs]

def get_top_tracks_list(artists, n=5):
    top_tracks = {}
    for artist in artists:
        top_tracks[artist] = get_top_tracks(artist, n)
    return top_tracks

def find_related_artists(name, max_related=5):
    data = sp.search(q=name)
    id = data["tracks"]["items"][0]["artists"][0]["id"]
    related = sp.artist_related_artists(id)
    related_names = [artist["name"] for artist in related["artists"]]
    
    print(related_names)
    return related_names[:int(max_related)]

def make_playlist(artists, max_related=5, max_songs=5):
    playlist = []
    for artist in artists:
        related_artists = find_related_artists(artist, max_related)
        tracks = get_top_tracks(artist, max_songs)
        playlist.extend(tracks)
        for related in related_artists:
            tracks = get_top_tracks(related, max_songs)
            playlist.extend(tracks)
    print(playlist)
    return playlist

top_tracks_list = get_top_tracks_list(artists, 5)

df = pd.DataFrame(top_tracks_list)
display(df)

related_artists_list = find_related_artists("Lhasa De Sala", 5)
display(related_artists_list)

playlist = make_playlist(artists, 5, 5)
display(playlist)

## Playlists

The `sp.featured_playlists()` method in `spotipy` fetches a list of Spotify's featured playlists at a given moment. These are curated playlists that Spotify often highlights on the platform's homepage. The method provides a snapshot of the playlists that are being promoted or featured by Spotify at the time of the request.

Once you've fetched the featured playlists, you can extract their IDs (and other details).

In [ ]:
featured = sp.featured_playlists() 
featured

### Getting a Playlist's Details
To fetch details about a specific playlist, you can use the playlist method. You'll need the playlist's Spotify ID.

In this example, we will use the following playlist id: *37i9dQZF1DXd9zR7tdziuQ*

In [72]:
playlist_id = featured['playlists']['items'][0]['id']
playlist = sp.playlist(playlist_id)

In [ ]:
print(playlist['name'])  # Print the playlist's name
print(playlist['description'])  # Print the playlist's description

### Getting Tracks from a Playlist
If you want to get the tracks from a specific playlist, you can use the playlist_tracks method.

In [ ]:
track_list = sp.playlist_tracks(playlist_id)
for track in track_list['items']:
    print(track['track']['name'])  # Print each track's name

### Getting Artists from a Playlist

To extract all the artists from the tracks in a playlist, you'd typically follow these steps:

1. Fetch the playlist's tracks.
2. Iterate through each track.
3. For each track, extract the associated artists.

In [ ]:
# List to store unique artist names
artists_list = []

for track_item in track_list['items']:
    track = track_item['track']
    for artist in track['artists']:
        artist_name = artist['name']
        if artist_name not in artists_list:  # This ensures each artist is added only once
            artists_list.append(artist_name)

print(artists_list)

## Exercise 2: Unraveling the World of Playlists


1. **Featured Exploration**: 
   - Fetch the list of Spotify's current featured playlists. 
   - Extract and display the names and IDs of the top 5 featured playlists.
   
2. **Deep Dive**:
   - Choose any one of the top 5 featured playlists (you can choose the one you personally find most interesting or simply pick one randomly).
   - Fetch and display its name, description, and total track count.

3. **Track-tastic**:
   - Extract and display the names of the first 10 tracks in the chosen playlist.

4. **Artistic Flair**:
   - Create a dictionary where the keys are the names of the first 10 tracks, and the values are lists containing the names of the artists associated with each track.
   - For example: `{"TrackName1": ["Artist1", "Artist2"], "TrackName2": ["Artist3"]}`
   

In [ ]:
featured_playlists = sp.featured_playlists(limit=5)

playlists_info = [{'name': playlist['name'], 'id': playlist['id']}
                  for playlist in featured_playlists['playlists']['items']]
display(pd.DataFrame(playlists_info))

playlist_info = sp.playlist('37i9dQZF1DWVuV87wUBNwc')
playlist_select_infos = {
    'name': playlist_info['name'], 'description': playlist_info['description'], 'tracks_count': playlist_info['tracks']['total']}

display(pd.DataFrame(playlist_select_infos, index=[0]))

playlist_tracks = sp.playlist_tracks('37i9dQZF1DWVuV87wUBNwc')['items']
track_names = [(track['track']['name']) for track in playlist_tracks][:10]
display(pd.DataFrame(track_names))

track_artist_dict = {track['track']['name']: [artist['name'] for artist in track['track']['artists']] for track in playlist_tracks[:10]}
print(json.dumps(track_artist_dict, indent=4))

